In [64]:
import os
import json
import glob
import pandas as pd
import numpy as np

In [65]:
RUN_ENV = 'PC' # 'PC' or 'COLAB'

# 0 Load Data by uploading json files maded by ning (need to repeat every time b4 u start)

In [66]:
if RUN_ENV == 'COLAB':
    from google.colab import files
    #json文件，地址在github库MF_KOC\Data\raw\, 导入上传文件为data frame格式
    uploaded_creator = files.upload()
    df_creator_0 = pd.read_json(next(iter(uploaded_creator)))
    df_creator = df_creator_0.copy()
    uploaded_content = files.upload()
    df_contents_0 = pd.read_json(next(iter(uploaded_content)))
    df_contents = df_contents_0.copy()
elif RUN_ENV == 'PC':
    #读取本地json文件
    df_creator = pd.read_json(r'..\Data\raw\creator_total.json')
    df_contents = pd.read_json(r'..\Data\raw\contents_total.json')

In [67]:
df_creator.shape

(102, 11)

In [68]:
df_contents.shape

(15941, 21)

# 1 Convert columns to the right dtype, and remove the irrelevant columns

In [69]:
# Define the desired data types for each column of df_creator
desired_dtypes = ['object']*6 + ['int32']*3 + ['object'] +  ['int64']
df = df_creator
# Assign the data types to the columns
for col, dtype in zip(df.columns, desired_dtypes):
    df[col] = df[col].astype(dtype)

In [70]:
# Define the desired data types for each column of df_contents
desired_dtypes = ['object']*5 +  ['int64'] + ['datetime64[s]'] + ['object']*3 +  ['int64']*4 + ['object']*3 + ['int64'] + ['object']*3
df = df_contents
# Assign the data types to the columns
for col, dtype in zip(df.columns, desired_dtypes):
    df[col] = df[col].astype(dtype)

# 2 Treat the df_contents table

### 7.2, 7.5, 7.6, 7.8, 7.9,

In [71]:
# remove columns, ['video_url', 'nickname', 'avatar', 'ip_location', 'note_url', 'source_keyword', 'xsec_token']
df_contents.drop(columns=['video_url', 'nickname', 'avatar', 'ip_location', 'note_url', 'source_keyword', 'xsec_token'], inplace=True)

### 7.1 Type: change the normal to 0, video to 1

In [72]:
df_contents['type'].value_counts()


type
normal    12415
video      3526
Name: count, dtype: int64

In [73]:
# Replace 'normal' with 0 and 'video' with 1 in the 'type' column
df_contents['type'] = df_contents['type'].map({'normal': 0, 'video': 1}).astype('int8')

# Change the column name 'type' to 'content_type_video'
df_contents.rename(columns={'type': 'content_type_video'}, inplace=True)

### 7.7 [Xinbo]Replace image urls with image count

In [74]:
# Create a new column 'image_count' by counting the URLs in 'image_list'
df_contents['image_count'] = df_contents['image_list'].apply(
    lambda x: len(x.split(',')) if isinstance(x, str) and x.strip() != '' else 0
)

# Drop the original 'image_list' column
df_contents.drop('image_list', axis=1, inplace=True)


In [75]:
# Convert 'image_count' to int64
df_contents['image_count'] = df_contents['image_count'].astype('int64')

### 7.12 [Xinbo]note的互动总数(帮助确定爆款笔记)

In [76]:
df_contents['interaction_count'] = df_contents[['liked_count', 'collected_count', 'comment_count']].sum(axis=1)


### 5. [Ning]筛选爆款hot note

In [77]:
# A. 对于千粉级creator(粉丝数>1000):
# 	If a note 互动总数+分享数 > 粉丝数/10 + 50 (用当前粉丝数, 忽略粉丝成长曲线)
# B. 对于素人级creator(1000>粉丝数):
# 	If a note 互动总数+分享数 > 150
def get_hot_note(df_creator,df_contents):
    """
    提取爆款笔记，并在原有完整笔记数据表中添加一列 hot_note, 用0或1标注是否为爆款笔记

    Args:
        df_creator (DataFrame): 完整用户数据.
        df_contents (DataFrame): 完整笔记数据.
    Return:
        df_hot_content (DataFrame): 爆款笔记数据
        df_contents (DataFrame): 添加后的笔记数据.
    """
    merged_df = pd.merge(
    df_creator[['user_id', 'fans']],
    df_contents,
    on='user_id'
    )
    maskA = (
        (merged_df['fans'] >= 1000) &
        ((merged_df['liked_count'] + merged_df['collected_count'] +
          merged_df['comment_count'] + merged_df['share_count']) >
         (merged_df['fans'] / 10 + 50))
    )

    maskB = (
        (merged_df['fans'] < 1000) &
        ((merged_df['liked_count'] + merged_df['collected_count'] +
          merged_df['comment_count'] + merged_df['share_count']) > 150)
    )

    mask = maskA | maskB
    hot_content = merged_df[mask]

    df_contents['hot_note'] = 0
    hot_list = hot_content['note_id'].values.tolist()
    for i in range(len(df_contents)):
      rows = df_contents.iloc[i]
      note_id = rows['note_id']
      if note_id in hot_list:
        df_contents.loc[df_contents['note_id']==note_id,'hot_note']=1

    return df_contents, hot_content

# Get hot notes and add a new column to df_contents
df_contents, df_hot_content = get_hot_note(df_creator, df_contents)

6.28 [Ning]新加average pictures per normal note, pic_per_normal_note 以及视频笔记占比, video_ratio\
6.29 [Ning]新加hot note count (请参考上面高亮的 5, 分两类creator, 但需要完成content表)\
6.30 [Ning]新加total share counts from hot notes ratio(同上)\
6.31 [Ning]新加最近一次发帖到抓取时的天数(floor取整), last_note2now\
6.32 [Ning]新加最近一次发hot note到抓取时的天数(floor取整), last_hot_note2now

In [78]:
def get_di(time):
      days = time / 86400000
      if days <= 25:
        d = np.exp(-0.02773*days)
      else:
        d = np.exp(-0.02773*25) * np.exp(-0.00396*(days-25))
      return d
def timestamp_diff_to_days(t1, t2):
    """
    计算毫米级时间戳的差值，并转化为天数，向下取整

    Args:
        t2 (int): 第二时间.
        t1 (int): 第一时间.
    Return:
        days (int): 差值天数
    """
    delta_ms = t2 - t1
    days = delta_ms // 86400000
    return days
def add_columns_creator_Ning(df_creator,df_contents,hot_content):
    """
    在用户数据中添加 hot_note_count, total_share_counts_hot_ratio, last_note2now, last_hot_note2now, pic_per_normal_note, video_ratio, weighted_total_share_counts

    Args:
        df_creator (DataFrame): 完整用户数据.
        df_contents (DataFrame): 完整笔记数据.
        hot_content (DataFrame): 爆款笔记数据.
    Return:
        df_creator (DataFrame): 添加完成后的用户数据
    """
    df_creator['pic_per_normal_note'] = 0.0
    df_creator['video_ratio'] = 0.0
    df_creator['hot_note_count'] = 0
    df_creator['total_share_counts_hot_ratio'] = 0
    df_creator['last_note2now'] = 0
    df_creator['last_hot_note2now'] = 0
    for i in range(len(df_creator)):
      num_video = 0
      num_image = 0
      num_share = 0
      rows = df_creator.iloc[i]
      user_id = rows['user_id']
      current_hot_content = hot_content[hot_content['user_id']== user_id]
      current_content = df_contents[df_contents['user_id']== user_id]
      for j in range(len(current_content)):
        rows = current_content.iloc[j]
        num_share += rows['share_count'] * get_di(timestamp_diff_to_days(rows['time'], rows['last_modify_ts']))
        note_type = rows['content_type_video']
        if note_type == 1:
          num_video += 1
        elif note_type == 0:
          num_image += rows['image_count']
      df_creator.loc[df_creator['user_id'] == user_id, 'weighted_total_share_counts'] = num_share
      df_creator.loc[df_creator['user_id'] == user_id, 'video_ratio'] = num_video/len(current_content)
      df_creator.loc[df_creator['user_id'] == user_id, 'pic_per_normal_note'] = min(num_image/(len(current_content)-num_video),8)
      df_creator.loc[df_creator['user_id'] == user_id, 'hot_note_count'] = len(current_hot_content)
      df_creator.loc[df_creator['user_id'] == user_id, 'total_share_counts_hot_ratio'] = current_hot_content['share_count'].sum()/current_content['share_count'].sum()
      time_last_note = df_contents.loc[df_contents['user_id']== user_id]['time'].max()
      time_craw_note = df_contents.loc[(df_contents['user_id']== user_id) & (df_contents['time']== time_last_note), 'last_modify_ts'].max()
      time_last_hot = hot_content.loc[hot_content['user_id']== user_id]['time'].max()
      time_craw_hot = hot_content.loc[(hot_content['user_id']== user_id) & (hot_content['time']== time_last_hot), 'last_modify_ts'].max()
      if len(current_hot_content) != 0:
        df_creator.loc[df_creator['user_id'] == user_id, 'last_note2now'] = int(timestamp_diff_to_days(time_last_note, time_craw_note))
        df_creator.loc[df_creator['user_id'] == user_id, 'last_hot_note2now'] = int(timestamp_diff_to_days(time_last_hot, time_craw_hot))
    return df_creator
df_contents, hot_content = get_hot_note(df_creator,df_contents.copy())
df_creator = add_columns_creator_Ning(df_creator,df_contents,hot_content)

C:\Users\Han\AppData\Local\Temp\ipykernel_11108\1618437053.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.558763816828333' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_creator.loc[df_creator['user_id'] == user_id, 'total_share_counts_hot_ratio'] = current_hot_content['share_count'].sum()/current_content['share_count'].sum()


### 7.13 [Yucheng]笔记在EDA图的时间戳可以设定为Time和last_update_time的一个函数。提取时间戳中的 年月日、小时、星期几、是否节假日 等特征

In [79]:
def clean_and_extract_time_features(df_contents):

    """
    统一原始时间字段格式，并添加基于发帖时间的时间特征。

    重命名字段（并统一为 datetime）：
        - time（发帖时间）：转换为 datetime
        - last_modify_ts → scraped_time（抓取时间）：转换并改名
        - last_update_time：已是 datetime 格式，做标准化
    """

    # 转换并重命名 last_modify_ts 为 scraped_time
    df_contents['scraped_time'] = pd.to_datetime(df_contents['last_modify_ts'], unit='ms')
    df_contents.drop(columns=['last_modify_ts'], inplace=True)

    # 转换其他时间字段
    df_contents['time'] = pd.to_datetime(df_contents['time'], unit='ms')

    # rename 'time' to 'post_time'
    df_contents.rename(columns={'time': 'post_time'}, inplace=True)

    return df_contents

df_contents = clean_and_extract_time_features(df_contents)

## Rearrange the columns in df_contents

In [80]:
# Define the desired column order
new_column_order = ['note_id', 'user_id', 'title', 'desc', 'tag_list', 'image_count',
                   'content_type_video', 'hot_note', 'post_time', 'last_update_time',
                   'scraped_time', 'liked_count', 'collected_count', 'comment_count',
                   'share_count', 'interaction_count']

# Reorder the columns using the new order
df_contents = df_contents[new_column_order]

# Change the desc to note_body
df_contents.rename(columns={'desc': 'note_body'}, inplace=True)

df_contents.head(2)

,note_id,user_id,title,note_body,tag_list,image_count,content_type_video,hot_note,post_time,last_update_time,scraped_time,liked_count,collected_count,comment_count,share_count,interaction_count
0,67d0d605000000002903d86a,576d3bde82ec3952ff40c5e1,有没有和我一样【吃茶三千】一喝一个不吱声的,连地址都不想写了[笑哭R]\n虽然环境还不错，颜值也不错\n但我感觉自己完全拔草了\n尝了m...,"达拉斯网红奶茶,达拉斯奶茶,达拉斯,达拉斯生活,达拉斯美食,达拉斯周边,达拉斯探店",1,0,0,2025-03-12 00:32:05,2025-03-12 00:32:06,2025-03-12 00:55:08.175,5,2,10,3,17
1,67c8960f000000002503ff5f,576d3bde82ec3952ff40c5e1,【达拉斯·玩】超治愈系手工体验！MUMU Garden,"📍301 W Parker Rd #208[话题]#, Plano, TX 75023（快乐...","208,达拉斯生活,达拉斯探店,达拉斯,达拉斯吃喝玩乐,达拉斯手工,达拉斯周末,达拉斯周边,...",14,0,0,2025-03-05 18:21:03,2025-03-05 18:21:04,2025-03-12 00:55:08.248,47,23,7,37,77


### 把df_conents 存为contents_cooked, 以后直接使用这个, 不要继续用df_conents

In [81]:
# save df_contents to contents_cooked.json
if RUN_ENV == 'COLAB':
    # Save the DataFrame exclude the columns ['year', 'month', 'dayofweek', 'is_weekend', 'hour'] to a JSON file in Google Colab
    df_contents.to_json(r'/content/contents_cooked.json', orient='records',date_format='iso', force_ascii=False)
elif RUN_ENV == 'PC':
    # Save the DataFrame exclude the columns ['year', 'month', 'dayofweek', 'is_weekend', 'hour'] to a JSON file on your local machine
    df_contents.to_json(r'..\Data\processed\contents_cooked.json', orient='records',date_format='iso', force_ascii=False)

# 3 Treat the df_creator table

## 6.6 - 6.10 Add Interaction Counts
Refine Creator (add sums)
- 6.6 Get the total amount of `liked`
- 6.7 Get the total amount of `collected`
- 6.8 Get the total amount of `comment`
- 6.9 Get the total amount of `shared`
- 6.10 Get the total number of posts as `note_count`
- The data type of added columns are set to integer

In [ ]:
# Ensure numeric columns in the content df are in the proper format
for col in ['liked_count', 'collected_count', 'comment_count', 'share_count']:
    df_contents[col] = pd.to_numeric(df_contents[col], errors='coerce')

# Group content df by user_id and sum the counts
agg_counts = df_contents.groupby('user_id')[['liked_count', 'collected_count', 'comment_count', 'share_count']].sum().reset_index()

# Merge the aggregated counts into the creator df by aligning on user_id
df_creator = pd.merge(df_creator, agg_counts, on='user_id', how='left')
# Convert the data type to integer
for col in ['liked_count', 'collected_count', 'comment_count', 'share_count']:
    df_creator[col] = df_creator[col].convert_dtypes()


NameError: name 'df_contents' is not defined

In [ ]:
note_counts = df_contents.groupby("user_id")["note_id"].count().reset_index(name="note_count")
df_creator = pd.merge(df_creator, note_counts, on="user_id", how="left")
df_creator["note_count"] = df_creator["note_count"].fillna(0).astype(int)

In [ ]:
df_creator.head(2)

,user_id,nickname,gender,avatar,desc,ip_location,follows,fans,interaction,tag_list,...,total_share_counts_hot_ratio,last_note2now,last_hot_note2now,weighted_total_share_counts,score_2,liked_count,collected_count,comment_count,share_count,note_count
0,576d3bde82ec3952ff40c5e1,达拉斯小鸡腿,女,https://sns-avatar-qc.xhscdn.com/avatar/61df4c...,慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰,美国,837,1653,11636,"{""info"": null, ""location"": ""美国"", ""profession"":...",...,0.558764,0,22,4432.999131,79.581669,6443,4951,2847,4433,146
1,58b325ce50c4b442ae017a30,小温柔Sylvia,女,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北京🇨🇳→ 达拉斯🇺🇸\n163cm ♋巨蟹座 乐观开朗 \nIG: sylviviishe...,美国,961,304,10699,"{""info"": null, ""location"": ""北京海淀""}",...,0.809131,5,5,1817.999749,47.761860,7290,3354,1713,1818,142


## Processing Tag list
There are four different categories in tag lists: info, location, profession and college.

Processing tag lists as
- Creating a columns as `zodiac` for 星座 。
- Note that there are 5 missing values in the `ip_location` column where the `location` column is not missing. I plan to use the `location` data to impute these missing `ip_location` values. Additionally, there are 22 cases where `ip_location` and `location` do not match (among the non-missing location entries), which include typos, missing values, and additional details present in the `location` field. (For detailed location info, we still need to discuss how to deal with it.)
- For `college`, `prefession`and `age`, there are only few KOC having this information, so I append it behind `desc` connected by '|'.

In [ ]:
import json
import pandas as pd

def extract_features(row):
    # Set default values when a field is missing
    zodiac = 'unknown'
    age = 'unknown'
    profession = 'unknown'
    college = 'unknown'

    try:
        tag_dict = json.loads(row['tag_list'])
    except Exception:
        # If tag_list is not valid JSON, return defaults.
        return pd.Series([zodiac, age, profession, college])

    # Process the 'info' field for zodiac and age.
    info = tag_dict.get('info')
    if info and isinstance(info, str):
        if '座' in info:
            zodiac = info  # if '座' is present, assume this is the zodiac info.
        if '岁' in info:
            age = info     # if '岁' is present, assume this is the age info.

    # Process 'profession'
    if tag_dict.get('profession'):
        profession = tag_dict.get('profession')

    # Process 'college'
    if tag_dict.get('college'):
        college = tag_dict.get('college')

    return pd.Series([zodiac, age, profession, college])

# Apply the extraction function to create a temporary DataFrame of extracted features.
extracted_features = df_creator.apply(extract_features, axis=1)
extracted_features.columns = ['zodiac', 'age', 'profession', 'college']

# Check how many rows have 'unknown' for each feature.
missing_counts = extracted_features.eq('unknown').sum()
print("Missing Values Counts:")
print(missing_counts)

# Optionally, inspect a few rows with missing zodiac information.
print("\nRows with unknown zodiac:")
print(extracted_features[extracted_features['zodiac'] == 'unknown'].head())


Missing Values Counts:
zodiac        74
age           91
profession    74
college       95
dtype: int64

Rows with unknown zodiac:
    zodiac      age profession  college
0  unknown  unknown       探店博主  unknown
1  unknown  unknown    unknown  unknown
2  unknown      35岁    unknown  unknown
3  unknown  unknown       美食博主  unknown
4  unknown  unknown    unknown  unknown


In [ ]:
def append_tags_to_desc(row):
    original_desc = row['desc']

    # Default values when a field is missing
    zodiac = 'unknown'
    age = 'unknown'
    profession = 'unknown'
    college = 'unknown'

    try:
        tag_dict = json.loads(row['tag_list'])
    except Exception:
        # If tag_list is not valid JSON, append unknown for all.
        appended = f"Zodiac: {zodiac} | Age: {age} | Profession: {profession} | College: {college}"
        return original_desc + " | " + appended

    # Process the 'info' field for zodiac and age.
    info = tag_dict.get('info')
    if info and isinstance(info, str):
        if '座' in info:
            zodiac = info
        if '岁' in info:
            age = info

    # Process 'profession'
    if tag_dict.get('profession'):
        profession = tag_dict.get('profession')

    # Process 'college'
    if tag_dict.get('college'):
        college = tag_dict.get('college')

    appended = f"Zodiac: {zodiac} | Age: {age} | Profession: {profession} | College: {college}"
    return original_desc + " | " + appended

# Update the 'desc' column in df_creator by applying the function row-wise.
df_creator['desc'] = df_creator.apply(append_tags_to_desc, axis=1)

In [ ]:
df_creator.head(2)

,user_id,nickname,gender,avatar,desc,ip_location,follows,fans,interaction,tag_list,...,total_share_counts_hot_ratio,last_note2now,last_hot_note2now,weighted_total_share_counts,score_2,liked_count,collected_count,comment_count,share_count,note_count
0,576d3bde82ec3952ff40c5e1,达拉斯小鸡腿,女,https://sns-avatar-qc.xhscdn.com/avatar/61df4c...,慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰 | Zodi...,美国,837,1653,11636,"{""info"": null, ""location"": ""美国"", ""profession"":...",...,0.558764,0,22,4432.999131,79.581669,6443,4951,2847,4433,146
1,58b325ce50c4b442ae017a30,小温柔Sylvia,女,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北京🇨🇳→ 达拉斯🇺🇸\n163cm ♋巨蟹座 乐观开朗 \nIG: sylviviishe...,美国,961,304,10699,"{""info"": null, ""location"": ""北京海淀""}",...,0.809131,5,5,1817.999749,47.761860,7290,3354,1713,1818,142


**location**

In [ ]:
# Parse the JSON strings in tag_list into dictionaries
tag_process = df_creator['tag_list'].apply(lambda x: json.loads(x))

# Extract the 'location' key for each row and assign it to a new 'location' column
df_creator['location'] = tag_process.apply(lambda x: x.get('location', None))

In [ ]:
df_creator['location'] = df_creator['location'].replace('UnitedStates','美国')
df_creator['location'] = df_creator['location'].replace('美国得克萨斯','美国TX')
df_creator['location'] = df_creator['location'].replace('UnitedStates...','美国')
df_creator['location'] = df_creator['location'].replace('海外美国','美国')

#### Note:
- ip_location: directly from profile page
- location: separated from user self-added tags
- 最后合并成一个ip_location: 三个等级, TX, US, CN

In [ ]:
df_creator[['ip_location','location']][((~df_creator['location'].isna()) & (df_creator['location'] != df_creator["ip_location"]))]

,ip_location,location
1,美国,北京海淀
5,美国,美国TX
20,,美国
30,美国,美国NY
31,墨西哥,美国TX
37,美国,美国TX
38,美国,美国TX
49,美国,巴巴多斯
54,美国,美国TX
55,,美国TX


### Impute `ip_location` by `location`

In [ ]:
df_creator['location'] = df_creator['location'].apply(
    lambda x: "TX" if isinstance(x, str) and "美国TX" in x
    else ("US" if isinstance(x, str) and "美国" in x else "abroad")
)


In [ ]:
df_creator['location'].head(10)

,location
0,US
1,abroad
2,US
3,US
4,US
5,TX
6,US
7,US
8,US
9,US


In [ ]:
df_creator = df_creator.drop(columns=['tag_list'])

## Feature Engineering

**KOC level**
- `ff_ratio`: ratio calculated by fans over follows
- `video_ratio`: ratio calculated by number of video posts over total number posts
- `account_age`: the time length in days between the newest post and oldest post

In [ ]:
df_creator['ff_ratio'] = df_creator.fans / df_creator.follows
time_length = df_contents.groupby('user_id') \
    .agg(oldest = ('post_time', 'min'), newest = ('post_time', 'max')).reset_index()
time_length['age_koc'] = time_length.newest - time_length.oldest
time_length['age_koc'] = time_length.age_koc.dt.days
df_creator = pd.merge(df_creator, time_length[['user_id', 'age_koc']], on = 'user_id')

In [ ]:
df_creator['score_2'].head()

,score_2
0,79.581669
1,47.761860
2,30.141722
3,35.930924
4,49.118844


In [ ]:
df_creator.describe()

In [ ]:
df_creator.isna().sum()

,0
user_id,0
nickname,0
gender,9
avatar,0
desc,0
ip_location,0
follows,0
fans,0
interaction,0
last_modify_ts,0


### Impute Gender

All female

In [ ]:
# Impute missing values with "女" and then map "女" to 1 and any other value to 0.
df_creator['is_female'] = df_creator['gender'].fillna("女").apply(lambda x: 1 if x == "女" else 0)

# Optionally, drop the original gender column
df_creator.drop('gender', axis=1, inplace=True)


In [ ]:
df_creator['is_female'].value_counts()

,count
is_female,
1,92
0,10


给KOC加post_span

In [ ]:
post_times = df_contents.groupby('user_id')['post_time'].agg(['min', 'max']).reset_index()
post_times['post_span'] = post_times['max'] - post_times['min']
print(post_times[['user_id', 'min', 'max', 'post_span']].head())


                    user_id                 min                 max  \
0  54d82c482e1d9366ebfc8ce5 2020-11-22 20:47:10 2025-03-08 05:22:29   
1  5530f8642e1d9365fa9a29fb 2021-05-10 16:54:57 2025-02-28 22:56:16   
2  55481451b203d97e86008c02 2020-12-02 00:14:57 2025-01-25 13:47:58   
3  556f1983f5a26347140b104b 2017-02-19 08:08:58 2025-03-13 22:58:53   
4  558a4a065894460336de85bf 2021-08-11 10:43:04 2025-03-16 04:03:26   

           post_span  
0 1566 days 08:35:19  
1 1390 days 06:01:19  
2 1515 days 13:33:01  
3 2944 days 14:49:55  
4 1312 days 17:20:22  


In [ ]:
df_creator = pd.merge(df_creator, post_times, on='user_id', how='left')
df_creator.head()

,user_id,nickname,avatar,desc,ip_location,follows,fans,interaction,last_modify_ts,liked_count,collected_count,comment_count,share_count,note_count,location,ff_ratio,is_female,min,max,post_span
0,576d3bde82ec3952ff40c5e1,达拉斯小鸡腿,https://sns-avatar-qc.xhscdn.com/avatar/61df4c...,慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰 | Zodi...,美国,837,1653,11636,1741740734045,6443,4951,2847,4433,146,US,1.974910,1,2019-07-27 23:00:43,2025-03-12 00:32:05,2054 days 01:31:22
1,58b325ce50c4b442ae017a30,小温柔Sylvia,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北京🇨🇳→ 达拉斯🇺🇸\n163cm ♋巨蟹座 乐观开朗 \nIG: sylviviishe...,美国,961,304,10699,1741746441631,7290,3354,1713,1818,142,abroad,0.316337,1,2018-06-12 07:38:23,2025-03-07 01:47:51,2459 days 18:09:28
2,5c0c067c0000000005022f0a,茄茄公主 🍆,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,坐标 达拉斯 美国🇺🇸 来美14 年 家庭成员： 三只毛小孩🐶 喜欢旅游 和探索各地美食🥰 ...,美国,251,258,3607,1741747540369,2344,1261,641,852,222,US,1.027888,1,2019-12-25 03:42:14,2025-02-15 04:23:29,1879 days 00:41:15
3,5649379603eb842b7ed00477,Sherry是睿睿呀,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北美生活探店| Chinese Girl in Dallas TX\n吃喝玩乐样样行 | 坐...,美国,1173,1326,28064,1741807240419,2567,1582,475,1211,66,US,1.130435,1,2023-11-15 07:27:31,2025-03-05 23:52:07,476 days 16:24:36
4,5d6a927e00000000010068b9,達拉斯里長,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,達拉斯，請多指教👋\n在朝九晚五平淡生活中，\n熱烈又燦爛的挖掘美好✨ | Zodiac: ...,美国,353,794,8149,1741814164296,3870,3753,452,1900,138,US,2.249292,1,2021-08-24 23:44:32,2025-02-07 22:07:07,1262 days 22:22:35


Add account length

In [ ]:
df_contents.columns

Index(['note_id', 'content_type_video', 'title', 'desc', 'post_time',
       'last_update_time', 'user_id', 'liked_count', 'collected_count',
       'comment_count', 'share_count', 'tag_list', 'image_count',
       'interaction_count', 'hot_note', 'extract_ts', 'year', 'month',
       'dayofweek', 'is_weekend', 'hour'],
      dtype='object')

In [ ]:
# Convert post_time in df_contents (assuming it's in seconds)
df_contents['post_time'] = pd.to_datetime(df_contents['post_time'], unit='s', errors='coerce')

# For df_creator, first inspect the values to check if they are in seconds or milliseconds.
print(df_creator['last_modify_ts'].head())

# If the values look too large (e.g., in the range of 13-digit numbers), they might be in milliseconds.
# Uncomment the appropriate conversion based on your inspection:

# If timestamps are in seconds:
# df_creator['last_modify_ts'] = pd.to_datetime(df_creator['last_modify_ts'], unit='s', errors='coerce')

# If timestamps are in milliseconds:
df_creator['last_modify_ts'] = pd.to_datetime(df_creator['last_modify_ts'], unit='ms', errors='coerce')

# Get the earliest post time for each user
earliest_posts = df_contents.groupby('user_id')['post_time'].min().reset_index()
earliest_posts.rename(columns={'post_time': 'first_post_time'}, inplace=True)

# Merge the earliest post time into df_creator based on user_id.
df_creator = pd.merge(df_creator, earliest_posts, on='user_id', how='left')

# Calculate account_length as the difference between last_modify_ts and first_post_time.
df_creator['account_length'] = df_creator['last_modify_ts'] - df_creator['first_post_time']

print(df_creator[['user_id', 'first_post_time', 'last_modify_ts', 'account_length']].head())


0    1741740734045
1    1741746441631
2    1741747540369
3    1741807240419
4    1741814164296
Name: last_modify_ts, dtype: int64
                    user_id     first_post_time          last_modify_ts  \
0  576d3bde82ec3952ff40c5e1 2019-07-27 23:00:43 2025-03-12 00:52:14.045   
1  58b325ce50c4b442ae017a30 2018-06-12 07:38:23 2025-03-12 02:27:21.631   
2  5c0c067c0000000005022f0a 2019-12-25 03:42:14 2025-03-12 02:45:40.369   
3  5649379603eb842b7ed00477 2023-11-15 07:27:31 2025-03-12 19:20:40.419   
4  5d6a927e00000000010068b9 2021-08-24 23:44:32 2025-03-12 21:16:04.296   

             account_length  
0 2054 days 01:51:31.045000  
1 2464 days 18:48:58.631000  
2 1903 days 23:03:26.369000  
3  483 days 11:53:09.419000  
4 1295 days 21:31:32.296000  


In [ ]:
df_creator

,user_id,nickname,avatar,desc,ip_location,follows,fans,interaction,last_modify_ts,liked_count,...,share_count,note_count,location,ff_ratio,is_female,min,max,post_span,first_post_time,account_length
0,576d3bde82ec3952ff40c5e1,达拉斯小鸡腿,https://sns-avatar-qc.xhscdn.com/avatar/61df4c...,慢慢了解世界，慢慢更新自己\n吃喝玩乐+宝宝们，随心分享\n佛系更新中～比心🫰 | Zodi...,美国,837,1653,11636,2025-03-12 00:52:14.045,6443,...,4433,146,US,1.974910,1,2019-07-27 23:00:43,2025-03-12 00:32:05,2054 days 01:31:22,2019-07-27 23:00:43,2054 days 01:51:31.045000
1,58b325ce50c4b442ae017a30,小温柔Sylvia,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北京🇨🇳→ 达拉斯🇺🇸\n163cm ♋巨蟹座 乐观开朗 \nIG: sylviviishe...,美国,961,304,10699,2025-03-12 02:27:21.631,7290,...,1818,142,abroad,0.316337,1,2018-06-12 07:38:23,2025-03-07 01:47:51,2459 days 18:09:28,2018-06-12 07:38:23,2464 days 18:48:58.631000
2,5c0c067c0000000005022f0a,茄茄公主 🍆,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,坐标 达拉斯 美国🇺🇸 来美14 年 家庭成员： 三只毛小孩🐶 喜欢旅游 和探索各地美食🥰 ...,美国,251,258,3607,2025-03-12 02:45:40.369,2344,...,852,222,US,1.027888,1,2019-12-25 03:42:14,2025-02-15 04:23:29,1879 days 00:41:15,2019-12-25 03:42:14,1903 days 23:03:26.369000
3,5649379603eb842b7ed00477,Sherry是睿睿呀,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,北美生活探店| Chinese Girl in Dallas TX\n吃喝玩乐样样行 | 坐...,美国,1173,1326,28064,2025-03-12 19:20:40.419,2567,...,1211,66,US,1.130435,1,2023-11-15 07:27:31,2025-03-05 23:52:07,476 days 16:24:36,2023-11-15 07:27:31,483 days 11:53:09.419000
4,5d6a927e00000000010068b9,達拉斯里長,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,達拉斯，請多指教👋\n在朝九晚五平淡生活中，\n熱烈又燦爛的挖掘美好✨ | Zodiac: ...,美国,353,794,8149,2025-03-12 21:16:04.296,3870,...,1900,138,US,2.249292,1,2021-08-24 23:44:32,2025-02-07 22:07:07,1262 days 22:22:35,2021-08-24 23:44:32,1295 days 21:31:32.296000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,5a5bf1414eacab4087b808c7,YSZ,https://sns-avatar-qc.xhscdn.com/avatar/6313b7...,分享生活 \n喜欢看演唱会 \n吃美食\n旅游\n 不定时更新🤓🤠 | Zodiac: un...,美国,1841,857,72730,2025-03-16 05:28:01.267,68637,...,1097,63,US,0.465508,1,2021-09-01 17:54:32,2025-02-23 22:45:01,1271 days 04:50:29,2021-09-01 17:54:32,1291 days 11:33:29.267000
98,621a0927000000001000f950,Esther 星星 ✨,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,生而自由 爱而无畏\n清醒知进退 酷而不失浪漫\n我是ENFP快乐小狗🐶\nIg：esthe...,美国,907,996,13953,2025-03-17 19:34:34.708,7465,...,2365,600,US,1.098126,1,2022-05-11 16:42:28,2025-03-14 05:23:25,1037 days 12:40:57,2022-05-11 16:42:28,1041 days 02:52:06.708000
99,5c4be6f500000000100050c6,小fafa,https://sns-avatar-qc.xhscdn.com/avatar/62f3f2...,📍TX🇺🇸\n吃喝｜遛娃｜摄影｜旅行\n旅行笔记已分类📁 | Zodiac: unknown...,美国,180,1732,30397,2025-03-19 18:03:18.300,14242,...,4286,372,US,9.622222,1,2020-04-09 19:48:27,2025-03-18 01:29:22,1803 days 05:40:55,2020-04-09 19:48:27,1804 days 22:14:51.300000
100,587623ba6a6a691cffba0875,苹果头在路上,https://sns-avatar-qc.xhscdn.com/avatar/1040g2...,｜35+ 会计打工人\n｜喜爱跳舞/追剧/分享生活有用信息/偶尔健身/男宝妈 | Zodia...,美国,94,575,5744,2025-03-19 23:27:28.370,3439,...,1398,301,US,6.117021,1,2023-07-20 07:05:34,2025-03-19 22:11:42,608 days 15:06:08,2023-07-20 07:05:34,608 days 16:21:54.370000


Add average post time & std

In [ ]:
import numpy as np
def compute_history_stats(times):
    if len(times) <= 1:
        # Return NaN for all four stats when not enough data
        return np.nan, np.nan, np.nan, np.nan
    # Ensure the times are sorted
    sorted_times = np.sort(times)
    # Compute intervals in seconds between posts
    intervals = np.diff(sorted_times).astype('timedelta64[s]').astype(float)
    avg_interval = intervals.mean()
    std_interval = intervals.std()

    # Convert times to numeric (seconds since epoch) for average and std computation
    numeric_times = times.astype('datetime64[s]').astype('int64')
    avg_post_numeric = numeric_times.mean()
    std_post_numeric = numeric_times.std()
    # Convert average back to datetime
    avg_post_time = np.datetime64(int(avg_post_numeric), 's')

    return avg_interval, std_interval, avg_post_time, std_post_numeric

# Group by user_id and apply the function on the 'post_time' column
history_stats = df_contents.groupby('user_id')['post_time'].apply(
    lambda x: compute_history_stats(x.values)
).reset_index()

# Expand the computed tuple into separate columns:
# history_avg: average interval between posts (in seconds)
# history_std: standard deviation of intervals (in seconds)
# post_avg: average posting time (datetime)
# post_std: standard deviation of post times (in seconds)
history_stats[['history_avg', 'history_std', 'post_avg', 'post_std']] = pd.DataFrame(
    history_stats['post_time'].tolist(), index=history_stats.index
)

# Remove the temporary column
history_stats.drop(columns=['post_time'], inplace=True)

# Merge the stats into df_creator based on user_id.
df_creator = pd.merge(df_creator, history_stats, on='user_id', how='left')

# Print out the results
print(df_creator[['user_id', 'history_avg', 'history_std', 'post_avg', 'post_std']].head())

# unit is in seconds


                    user_id   history_avg   history_std            post_avg  \
0  576d3bde82ec3952ff40c5e1  1.223938e+06  4.806686e+06 2022-11-17 15:53:58   
1  58b325ce50c4b442ae017a30  1.507255e+06  5.636074e+06 2023-02-27 05:37:24   
2  5c0c067c0000000005022f0a  7.346067e+05  1.077335e+06 2021-11-11 08:53:28   
3  5649379603eb842b7ed00477  6.336227e+05  1.657265e+06 2024-10-21 08:26:10   
4  5d6a927e00000000010068b9  7.964770e+05  2.409343e+06 2022-12-14 11:53:24   

       post_std  
0  3.297572e+07  
1  4.406148e+07  
2  4.007945e+07  
3  7.186721e+06  
4  3.483827e+07  


Add counts within 90 days

In [ ]:
import pandas as pd

# Ensure timestamps are in datetime format
df_creator['last_modify_ts'] = pd.to_datetime(df_creator['last_modify_ts'])
df_contents['post_time'] = pd.to_datetime(df_contents['post_time'])

# Merge last_modify_ts into df_contents for each note (to use the creator's timestamp in filtering)
df_contents = df_contents.merge(
    df_creator[['user_id', 'last_modify_ts']],
    on='user_id',
    how='left',
    suffixes=('', '_creator')
)

# Filter for notes posted within 90 days before (or on) the creator's last_modify_ts
df_recent = df_contents[
    df_contents['post_time'] >= df_contents['last_modify_ts'] - pd.Timedelta(days=90)
]

# Group by user_id and sum the counts from recent notes.
grouped_recent = df_recent.groupby('user_id').agg(
    liked_90=('liked_count', 'sum'),
    comment_90=('comment_count', 'sum'),
    collected_90=('collected_count', 'sum'),
    note_count_90=('note_id', 'count')  # counting the number of notes
)

# If df_contents has a 'share_count' column, include it in the aggregation:
if 'share_count' in df_contents.columns:
    grouped_recent['share_90'] = df_recent.groupby('user_id')['share_count'].sum()

# Merge the recent counts back into df_creator.
df_creator = pd.merge(df_creator, grouped_recent, on='user_id', how='left')

# Replace any missing values (users with no notes in the past 90 days) with 0.
df_creator[['liked_90', 'comment_90', 'collected_90', 'note_count_90']] = \
    df_creator[['liked_90', 'comment_90', 'collected_90', 'note_count_90']].fillna(0)

if 'share_90' in df_creator.columns:
    df_creator['share_90'] = df_creator['share_90'].fillna(0)

print(df_creator[['user_id', 'liked_90', 'comment_90', 'collected_90', 'share_90' if 'share_90' in df_creator.columns else '', 'note_count_90']].head())


                    user_id  liked_90  comment_90  collected_90  share_90  \
0  576d3bde82ec3952ff40c5e1     330.0        92.0         203.0     289.0   
1  58b325ce50c4b442ae017a30     565.0       207.0         526.0     336.0   
2  5c0c067c0000000005022f0a      15.0         4.0           0.0       1.0   
3  5649379603eb842b7ed00477     533.0        61.0         124.0      84.0   
4  5d6a927e00000000010068b9     127.0        27.0          55.0      50.0   

   note_count_90  
0            8.0  
1           20.0  
2            3.0  
3           17.0  
4            6.0  


In [ ]:

threshold_ms = 90 * 24 * 3600 * 1000  # 90 days in ms

# Merge df_contents with the subset of df_creator that has user_id and last_modify_ts.
# Use suffixes so that the creator's timestamp comes out with a unique name.
df_merged = pd.merge(
    df_contents,
    df_creator[['user_id', 'last_modify_ts']],
    on='user_id',
    how='left',
    suffixes=('', '_creator')
)

print("Columns in df_merged:", df_merged.columns.tolist())

# Now, the reference timestamp should be in the column 'last_modify_ts_creator'
# Filter for posts within the 90-day window from each creator's last_modify_ts.
df_recent = df_merged[df_merged['time'] >= (df_merged['last_modify_ts_creator'] - threshold_ms)]

# Aggregate the metrics for these recent posts by user_id.
metrics_90 = df_recent.groupby('user_id').agg(
    liked_90=('liked_count', 'sum'),
    collected_90=('collected_count', 'sum'),
    comment_90=('comment_count', 'sum'),
    share_90=('share_count', 'sum'),
    note_count_90=('note_id', 'count')
).reset_index()

# Merge the 90-day metrics into df_creator.
df_creator = pd.merge(df_creator, metrics_90, on='user_id', how='left')

# For creators with no posts in the last 90 days, fill missing values with 0.
cols_to_fill = ['liked_90', 'collected_90', 'comment_90', 'share_90', 'note_count_90']
df_creator[cols_to_fill] = df_creator[cols_to_fill].fillna(0)

# Display the new columns to verify the result.
print(df_creator[['user_id', 'liked_90', 'collected_90', 'comment_90', 'share_90', 'note_count_90']].head())


Columns in df_merged: ['note_id', 'content_type_video', 'title', 'desc', 'post_time', 'last_update_time', 'user_id', 'liked_count', 'collected_count', 'comment_count', 'share_count', 'tag_list', 'image_count', 'interaction_count', 'hot_note', 'extract_ts', 'year', 'month', 'dayofweek', 'is_weekend', 'hour', 'last_modify_ts_creator']


KeyError: 'time'

In [ ]:
import pandas as pd

# Ensure the timestamps are in datetime format.
df_creator['last_modify_ts'] = pd.to_datetime(df_creator['last_modify_ts'])
df_contents['post_time'] = pd.to_datetime(df_contents['post_time'])

# Merge the creator's last_modify_ts into df_contents.
df_contents = df_contents.merge(
    df_creator[['user_id', 'last_modify_ts']],
    on='user_id',
    how='left',
    suffixes=('', '_creator')
)

# Filter for notes posted within 180 days before (or on) the creator's last_modify_ts.
df_recent_180 = df_contents[
    df_contents['post_time'] >= df_contents['last_modify_ts'] - pd.Timedelta(days=180)
]

# Group by user_id and aggregate the counts.
grouped_recent_180 = df_recent_180.groupby('user_id').agg(
    liked_180=('liked_count', 'sum'),
    comment_180=('comment_count', 'sum'),
    collected_180=('collected_count', 'sum'),
    note_count_180=('note_id', 'count')
)

# If the 'share_count' column exists in df_contents, include it:
if 'share_count' in df_contents.columns:
    grouped_recent_180['share_180'] = df_recent_180.groupby('user_id')['share_count'].sum()

# Merge the aggregated counts back into df_creator.
df_creator = pd.merge(df_creator, grouped_recent_180, on='user_id', how='left')

# Replace missing values with 0 (for creators with no notes in the past 180 days).
for col in ['liked_180', 'comment_180', 'collected_180', 'note_count_180']:
    df_creator[col] = df_creator[col].fillna(0)

if 'share_180' in df_creator.columns:
    df_creator['share_180'] = df_creator['share_180'].fillna(0)

# Display the updated columns for verification.
print(df_creator[['user_id', 'liked_180', 'comment_180', 'collected_180', 'note_count_180', 'share_180']].head())


                    user_id  liked_180  comment_180  collected_180  \
0  576d3bde82ec3952ff40c5e1      728.0        219.0          476.0   
1  58b325ce50c4b442ae017a30      598.0        328.0          539.0   
2  5c0c067c0000000005022f0a       67.0          5.0           44.0   
3  5649379603eb842b7ed00477     1555.0        299.0         1045.0   
4  5d6a927e00000000010068b9      334.0         69.0          186.0   

   note_count_180  share_180  
0            13.0      687.0  
1            27.0      356.0  
2             6.0       43.0  
3            49.0      887.0  
4            14.0      210.0  
